In [54]:
DATA_FILE = "scout_outfield.csv" 
MODEL_FILE = "knn_similarity.pkl"
SCALER_FILE = "knn_scaler.pkl"

In [48]:
import pandas as pd
import numpy as np
import pickle
import json
import os
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# --- CONFIGURATION ---
DATA_FILE = "scout_outfield.csv"
MODEL_FILE = "knn_similarity.pkl"
SCALER_FILE = "knn_scaler.pkl"
PCA_FILE = "knn_pca.pkl"          
FEATURE_FILE = "dna_features.json"

# --- 1. PRECISE FEATURE SELECTION ---
DNA_FEATURES = [
    # SCORING & THREAT
    'npxG_Per', 'Sh_per_90_Standard', 'SoT_percent_Standard', 'Att Pen_Touches_Per90',
    
    # CREATION
    'PrgP_Per90', 'KP_Per90', 'xA_Expected', 'Cmp_percent_Total', 'Cmp_Long_Per90',
    'Final_Third_Per90', 'PPA_Per90', 'CrsPA_Per90',
    
    # DRIBBLING / CARRYING
    'PrgC_Carries_Per90', 'PrgDist_Carries_Per90', 'Succ_Take_Per90', 'Dis_Carries',
    
    # DEFENSE 
    'TklW_Tackles_Per90', 'Int_Def_Per90', 'Blocks_Blocks_Per90', 
    'Won_percent_Aerial', 'Recov_Per90', 'Clr_Per90'
]

# --- 2. TRAINING PIPELINE ---
if os.path.exists(DATA_FILE):
    print("📂 Loading Scout Data...")
    df = pd.read_csv(DATA_FILE)
    
    # Select only existing columns
    active_features = [c for c in DNA_FEATURES if c in df.columns]
    print(f"🧬 Selected {len(active_features)} Features.")

    # A. SCALING 
    scaler = RobustScaler()
    X = df[active_features].fillna(0)
    X_scaled = scaler.fit_transform(X)

    # B. PCA 
    print("📉 Applying PCA (Dimensionality Reduction)...")
    pca = PCA(n_components=0.95) 
    X_pca = pca.fit_transform(X_scaled)
    print(f"   Reduced dimensions from {X_scaled.shape[1]} to {X_pca.shape[1]}")

    # C. KNN 
    print("🧠 Training KNN Engine...")
    knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
    knn.fit(X_pca)

    # --- SAVE ---
    with open(MODEL_FILE, 'wb') as f: pickle.dump(knn, f)
    with open(SCALER_FILE, 'wb') as f: pickle.dump(scaler, f)
    with open(PCA_FILE, 'wb') as f: pickle.dump(pca, f) # Save PCA
    with open(FEATURE_FILE, 'w') as f: json.dump(active_features, f)
        
    print("✅ Model Trained with PCA & Saved!")

📂 Loading Scout Data...
🧬 Selected 22 Features.
📉 Applying PCA (Dimensionality Reduction)...
   Reduced dimensions from 22 to 17
🧠 Training KNN Engine...
✅ Model Trained with PCA & Saved!


In [50]:
import pandas as pd
import numpy as np
import pickle
import json
import os

# CONFIG
DATA_FILE = "scout_outfield.csv"
MODEL_FILE = "knn_similarity.pkl"
SCALER_FILE = "knn_scaler.pkl"
PCA_FILE = "knn_pca.pkl"         
FEATURE_FILE = "dna_features.json"

# --- TACTICAL OVERLAP MAP ---
OVERLAP_MAP = {
    'CB':  {'CB': 1.0, 'CDM': 0.5, 'RB': 0.2, 'LB': 0.2},
    'LB':  {'LB': 1.0, 'RB': 1.0, 'LWB': 1.0, 'LM': 0.5, 'CB': 0.2},
    'RB':  {'RB': 1.0, 'LB': 1.0, 'RWB': 1.0, 'RM': 0.5, 'CB': 0.2},
    'CDM': {'CDM': 1.0, 'CM': 0.8, 'CB': 0.5},
    'CM':  {'CM': 1.0, 'CDM': 0.8, 'CAM': 0.6},
    'CAM': {'CAM': 1.0, 'CM': 0.6, 'LW': 0.4, 'RW': 0.4, 'ST': 0.2},
    'LM':  {'LM': 1.0, 'RM': 1.0, 'LW': 0.8, 'LB': 0.5},
    'RM':  {'RM': 1.0, 'LM': 1.0, 'RW': 0.8, 'RB': 0.5},
    'LW':  {'LW': 1.0, 'RW': 1.0, 'LM': 0.8, 'ST': 0.4, 'CAM': 0.4},
    'RW':  {'RW': 1.0, 'LW': 1.0, 'RM': 0.8, 'ST': 0.4, 'CAM': 0.4},
    'ST':  {'ST': 1.0, 'CF': 1.0, 'LW': 0.4, 'RW': 0.4},
}

def evaluate_smart_accuracy():
    print("🧪 STARTING SMART EVALUATION...")
    
    # 1. Load Resources
    try:
        df = pd.read_csv(DATA_FILE)
        with open(MODEL_FILE, 'rb') as f: knn = pickle.load(f)
        with open(SCALER_FILE, 'rb') as f: scaler = pickle.load(f)
        with open(PCA_FILE, 'rb') as f: pca = pickle.load(f)
        with open(FEATURE_FILE, 'r') as f: features = json.load(f)
    except FileNotFoundError:
        print("❌ Files missing. Run training first.")
        return

    # 2. Process Data
    X = df[features].fillna(0)
    X_scaled = scaler.transform(X)
    X_final = pca.transform(X_scaled) 

    # 3. Test Loop
    if 'main_pos' not in df.columns:
        print("❌ 'main_pos' column missing.")
        return

    valid_df = df.dropna(subset=['main_pos'])
    sample = valid_df.sample(min(500, len(valid_df)), random_state=42)
    
    total_score = 0
    total_neighbors = 0
    perfect_matches = 0
    
    print(f"   Analyzing {len(sample)} players using Tactical Overlap Map...")
    
    for idx, row in sample.iterrows():
        # Get Target Pos
        target_pos = str(row['main_pos']).split(',')[0].strip()
        
        # Validate target pos exists in map
        if target_pos not in OVERLAP_MAP: continue
        
        # Get Neighbors
        mat_idx = df.index.get_loc(idx)
        
        # --- FIXED LINES BELOW ---
        dists, inds = knn.kneighbors([X_final[mat_idx]], n_neighbors=6)
        
        # Check Neighbors (Skip self at index 0)
        for i in range(1, len(inds[0])): 
            n_idx = inds[0][i]
            n_pos = str(df.iloc[n_idx]['main_pos']).split(',')[0].strip()
            
            # --- SCORING ---
            points = 0
            
            # 1. Direct Lookup in Map
            allowed = OVERLAP_MAP.get(target_pos, {})
            if n_pos in allowed:
                points = allowed[n_pos]
            
            # 2. Fallback: Exact String Match
            elif n_pos == target_pos:
                points = 1.0

            total_score += points
            total_neighbors += 1
            if points == 1.0: perfect_matches += 1

    # 4. Results
    if total_neighbors == 0:
        print("⚠️ No valid neighbors analyzed.")
        return

    avg_score = (total_score / total_neighbors) * 100
    perfect_rate = (perfect_matches / total_neighbors) * 100
    
    print("-" * 40)
    print(f"✅ TACTICAL CONSISTENCY SCORE: {avg_score:.1f}%")
    print(f"🔹 Perfect Position Matches:     {perfect_rate:.1f}%")
    print("-" * 40)
    
    if avg_score > 75: print("   🚀 EXCELLENT: The model understands player roles.")
    elif avg_score > 60: print("   ✅ ACCEPTABLE: Good logic, some hybrid overlaps.")
    else: print("   ⚠️ POOR: Still confusing distinct roles.")

evaluate_smart_accuracy()

🧪 STARTING SMART EVALUATION...
   Analyzing 500 players using Tactical Overlap Map...
----------------------------------------
✅ TACTICAL CONSISTENCY SCORE: 66.4%
🔹 Perfect Position Matches:     54.8%
----------------------------------------
   ✅ ACCEPTABLE: Good logic, some hybrid overlaps.


In [56]:
import pandas as pd
import pickle
import json
import os

# CONFIG
DATA_FILE = "scout_outfield.csv"
MODEL_FILE = "knn_similarity.pkl"
SCALER_FILE = "knn_scaler.pkl"
FEATURE_FILE = "dna_features.json" # <--- The file we just saved

def sanity_check():
    test_cases = ["Erling Haaland", "Trent Alexander-Arnold", "Kevin de bruyne"]
    
    # 1. Load Data
    df = pd.read_csv(DATA_FILE)
    df['search_key'] = df['Player'].astype(str).str.lower().str.strip()
    
    # 2. Load Model, Scaler AND Feature List
    try:
        with open(MODEL_FILE, 'rb') as f: knn = pickle.load(f)
        with open(SCALER_FILE, 'rb') as f: scaler = pickle.load(f)
        with open(FEATURE_FILE, 'r') as f: saved_features = json.load(f) # <--- Load the list
    except FileNotFoundError:
        print("❌ Error: Missing model files. Run the training script first.")
        return

    # 3. Prepare Data Matrix
    X_scaled = scaler.transform(df[saved_features].fillna(0))

    print("\n🧐 RUNNING SANITY CHECKS...")
    
    for player in test_cases:
        s_key = player.lower().strip()
        found = df[df['search_key'] == s_key]
        
        if found.empty:
            print(f"⚠️ Skipped {player} (Not in database)")
            continue
            
        idx = found.index[0]
        mtx_idx = df.index.get_loc(idx)
        
        # Predict
        dists, inds = knn.kneighbors([X_scaled[mtx_idx]], n_neighbors=4)
        
        # Print Results
        p_row = found.iloc[0]
        pos = p_row['main_pos'] if 'main_pos' in p_row else p_row.get('Pos', '?')
        
        print(f"\nPlayer: {p_row['Player']} ({pos})")
        print(f"Similar to:")
        for i in range(1, 4):
            n_idx = inds[0][i]
            neighbor = df.iloc[n_idx]
            n_pos = neighbor['main_pos'] if 'main_pos' in neighbor else neighbor.get('Pos', '?')
            
            # Convert cosine distance to similarity %
            sim_score = (1 - dists[0][i]) * 100
            print(f"   -> {neighbor['Player']} ({n_pos}) - {sim_score:.1f}% Match")

sanity_check()


🧐 RUNNING SANITY CHECKS...

Player: erling haaland (ST)
Similar to:
   -> alvaro morata (ST) - 95.9% Match
   -> goncalo ramos (ST) - 95.6% Match
   -> robert lewandowski (ST) - 95.5% Match

Player: trent alexander-arnold (RB)
Similar to:
   -> hakan calhanoglu (CDM) - 97.5% Match
   -> oscar mingueza (RB) - 96.6% Match
   -> joshua kimmich (CDM) - 96.2% Match

Player: kevin de bruyne (CM)
Similar to:
   -> alex baena (CAM) - 95.6% Match
   -> james maddison (CM) - 94.9% Match
   -> michael olise (RM) - 94.1% Match
